In [38]:
from langhuan import NERTask
from langhuan.utility import findall_word_position
from forgebox.imports import *
from sklearn.datasets import fetch_20newsgroups
from uuid import uuid4
import regex

In [2]:
news20 = fetch_20newsgroups()
df = pd.DataFrame(dict(text=news20['data']))
df.head()

,text
0,From: lerxst@wam.umd.edu (where's my thing)\nS...
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...


In [3]:
def word_finder(kw, label):
    def get_tags(text):
        tags = []
        for offset in findall_word_position(text, kw):
            tags.append(dict(
                text=text[offset:offset+len(kw)],
                label=label,
                offset=offset
            ))
        return dict(tags=tags, preset="preset")
    return get_tags

## Set extra unique id

In [4]:
df["special_id"] = list(str(uuid4()) for uid in range(len(df)))
df = df.set_index("special_id")

## Preset some tags

In [5]:
df["guess"] = df.text.apply(word_finder("university","institution"))

In [6]:
df

,text,guess
special_id,,
f7ec2c6f-046a-469f-a18a-d4a0540af07b,From: lerxst@wam.umd.edu (where's my thing)\nS...,"{'tags': [{'text': 'University', 'label': 'ins..."
4c2eecd7-39f4-4d7c-b586-a7c7803a74c0,From: guykuo@carson.u.washington.edu (Guy Kuo)...,"{'tags': [{'text': 'University', 'label': 'ins..."
a96d71fa-47d4-474a-b835-6078761865d7,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,"{'tags': [{'text': 'University', 'label': 'ins..."
acd720f6-d70b-457b-ac1c-fb175f91bd9e,From: jgreen@amber (Joe Green)\nSubject: Re: W...,"{'tags': [], 'preset': 'preset'}"
c34a56dc-a053-4cfe-8606-498087d49e79,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,"{'tags': [], 'preset': 'preset'}"
...,...,...
47d62545-3a21-402c-83a6-7a590981d06c,From: jim.zisfein@factory.com (Jim Zisfein) \n...,"{'tags': [], 'preset': 'preset'}"
829cde95-4d2d-4b85-ad15-0d062832b4dc,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,"{'tags': [{'text': 'University', 'label': 'ins..."
3b96eff1-8ba6-4d98-9c11-49e661230a20,From: westes@netcom.com (Will Estes)\nSubject:...,"{'tags': [], 'preset': 'preset'}"


## Run service

### Service with preset tags

In [7]:
app = NERTask.from_df(df, text_col="text", options=["institution", "company", "name"], preset_tag_col="guess")

### Service without preset tags

In [10]:
app = NERTask.from_df(df, text_col="text", options=["institution", "company", "name"])

In [11]:
app.run("0.0.0.0", port=5001)

 * Serving Flask app "task_NER_210901_105527" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Sep/2021 10:55:38] "GET /admin HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 10:55:38] "POST /get_options HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 10:55:38] "POST /stats HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 10:55:42] "GET /?index=3 HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 10:55:43] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 10:55:43] "GET /personal_history?user_id=2fe81aa184 HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 10:55:56] "POST /tagging HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 10:55:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 10:55:56] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 10:55:56] "GET /personal_history?user_id=2fe81aa184 HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 10:56:01] "POST /data HTTP/1.1" 200 -
